# HMMLearn Experimentation
## Here is a list of all the things that we ave during sensing

1st sense you have comes from,

### handle_opponent_move_result
- captured_piece = bool
- captured_square = chess.Square

### sense result
- 3 x 3 sese result tuples
Each square is (coordinate, piece type, player type (white or black)

## can you see what piece took your piece?


## The key point is that you have all these senses *before a move is chosen*


# How can we vectorize this information into a standardized emission. Non one hot encoding

## emission will look like the following
    [int,          -> captured piece   . 0 if captured 1 if not captured
    
    int,           -> captured square 0, 0 if nothing was captured. (Else 1 - 8)
    int,                                 0 if nothing was captured  (Else 1 - 8)
    
    int,            -> this represents the center coordinate of our sense (1 - 8), (1 - 8)
    int,
    
    int,            -> this repesents the piece type for square 1, 1 of our sense result. 0 for None, 1 - 6 for piece type 
    int,            -> 0 if white, 1 if black
    
    int,            -> this repesents the piece type for square 2, 1 of our sense result. 0 for None, 1 - 6 for piece type 
    int,            -> 0 if white, 1 if black
    
    ...
    
    int,            -> this repesents the piece type for square 3, 3 of our sense result. 0 for None, 1 - 6 for piece type 
    int,            -> 0 if white, 1 if black    
    ]
    

    Number of features in emmision = 5 + 2 * 9 = 23.
    
   Note: a one hot encoding might be better


# Okay Now Lets do some coding

In [9]:
from hmmlearn import hmm
import numpy as np

In [10]:
model = hmm.GaussianHMM(n_components=30)  # I think n components litt

# Okay What does this and how can we use it


In [3]:
# lets say that white and black have a bunch of moves. White moves 25 times and black moves 24 times. This ends in a white win

white_model = hmm.GaussianHMM(n_components = 3, covariance_type="diag") # how many compoenents is this
black_model = hmm.GaussianHMM(n_components = 3, covariance_type="diag")

# A GAme
sample_data_white = np.random.randint(0, 6, (25, 23))
sample_data_black = np.random.randint(0, 6, (24, 23))



In [11]:
# lets fir the hmm on this data
white_model.fit(sample_data_white)
black_model.fit(sample_data_black)
# NOTE: I am fully aware that this isnt enough data to train with

Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'


GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
            covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=3, n_iter=10, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=False)

# Now lets say that we have trained model and we need to use it to predict the state for a current game

Let's say we are on the 10th move playing as white. (White has recieved observations but has not moved yet. We are now in choose move and need to guess our state



In [12]:
historical_observations = np.random.randint(0, 6, (10, 23))

hidden_states_probs = white_model.predict_proba(historical_observations) # we have three states in total
print(hidden_states_probs)
print(hidden_states_probs.shape) # I think we might be able to extract more state information per timestep 

[[9.67263773e-27 1.00000000e+00 3.22505922e-62]
 [1.20942778e-03 8.98254994e-01 1.00535578e-01]
 [4.14443367e-04 9.99585557e-01 6.12646249e-42]
 [2.07872786e-04 9.99792127e-01 7.34543489e-16]
 [3.31421890e-02 9.66857811e-01 2.91715980e-16]
 [3.95898547e-02 9.60410145e-01 2.16998779e-17]
 [3.98658803e-01 6.01341197e-01 2.07627154e-22]
 [1.78648802e-05 9.70546972e-05 9.99885080e-01]
 [8.01777661e-01 1.56168619e-01 4.20537204e-02]
 [9.69404994e-01 3.05950064e-02 4.12131896e-31]]
(10, 3)


# What is this output saying
    ## there is a row for every timestep. Every row sums to one.
so hidden_states_probs[i, j] is the probability of being in state j at time step i

# Okay so here is the question... How do we train this so it does not suck 
## My solution is to run N games (each with a varying length) with a random agent playing against another random agent. We then fit this model on all of the games


In [6]:
"""
psuedocode
white_obs = []
black_obs = []
for i in range(N):
    play game
    white_obs.append(Sequence of Observations for white last game)
    black_obs.append(Seuence of Observations for black last game)
  
"""

def play_game():
    """
    Function that returns white obs, black obs

    
    """
    game_length= np.random.randint(50, 60)
    if np.random.randint(0, 1) == 0:
        sample_data_white = np.random.randint(0, 6, (game_length, 23)) # person who moves has one more
        sample_data_black = np.random.randint(0, 6, (game_length - 1, 23))    
    else:
        sample_data_white = np.random.randint(0, 6, (game_length, 23))
        sample_data_black = np.random.randint(0, 6, (game_length, 23))
    return sample_data_white, sample_data_black

In [14]:
N = 100
white_model = hmm.GaussianHMM(n_components = 3, covariance_type="diag") # how many compoenents is this
black_model = hmm.GaussianHMM(n_components = 3, covariance_type="diag")
white_obs_list, black_obs_list = [], []
for i in range(N):
    white_obs, black_obs = play_game()
    white_obs_list.append(white_obs)
    black_obs_list.append(black_obs)
    

# train models
lengths_of_sequences_white = [len(i) for i in white_obs_list]
lengths_of_sequences_black = [len(i) for i in black_obs_list]



white_obs_concat = np.vstack(white_obs_list)
black_obs_concat = np.vstack(black_obs_list)
white_model.fit(white_obs_concat, lengths_of_sequences_white)
black_model.fit(black_obs_concat, lengths_of_sequences_black)
    

GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
            covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=3, n_iter=10, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=False)

In [8]:
print(white_model)

GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
            covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=3, n_iter=10, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=False)
